# 📊 Análisis Exploratorio de Ventas - Sistema AurelionStream2

## 🎯 Descripción General

Este notebook documenta el sistema completo de análisis exploratorio de datos de ventas desarrollado para **AurelionStream2**. El análisis procesa datos de múltiples archivos Excel para generar insights profundos sobre patrones de ventas, comportamiento por categorías de productos, correlaciones y detección de outliers.

## 📁 Estructura de Datos
- **`clientes.xlsx`** → Información de clientes
- **`detalle_ventas.xlsx`** → Transacciones detalladas
- **`productos.xlsx`** → Catálogo de productos con categorías
- **`ventas.xlsx`** → Información de ventas principales

## 🎯 Objetivos del Análisis
1. **Estadísticas Descriptivas** completas por variable
2. **Análisis por Categorías** de productos (Alimentos, Bebidas, Limpieza)
3. **Correlaciones** entre precio, cantidad y total de ventas
4. **Detección de Outliers** usando método IQR
5. **Visualizaciones Avanzadas** con múltiples gráficos
6. **Generación de Reportes** automáticos

# 1. 📦 Importación de Librerías Necesarias

El sistema requiere las siguientes librerías para el análisis completo:

In [ ]:
# Librerías para manejo de rutas y sistema
from pathlib import Path
import os
import sys
import unicodedata

# Librerías para análisis de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de rutas del proyecto
PROJECT_ROOT = Path(__file__).resolve().parents[1] if '__file__' in globals() else Path.cwd()
DB_DIR = PROJECT_ROOT / "DataBase"
OUT_DIR = PROJECT_ROOT / "outputs"
FIG_DIR = OUT_DIR / "figures"

print("✅ Librerías importadas correctamente")
print(f"📁 Directorio del proyecto: {PROJECT_ROOT}")
print(f"📊 Directorio de datos: {DB_DIR}")
print(f"📈 Directorio de resultados: {OUT_DIR}")

# 2. 🔧 Funciones Utilitarias

## 2.1 Normalización de Nombres de Columnas

La función `normalize_col_name()` estandariza los nombres de columnas para evitar problemas con acentos, espacios y mayúsculas:

In [ ]:
def normalize_col_name(s: str) -> str:
    """
    Normaliza nombres de columnas eliminando acentos, 
    convirtiendo a minúsculas y reemplazando espacios por guiones bajos.
    
    Args:
        s (str): Nombre de columna a normalizar
        
    Returns:
        str: Nombre normalizado
    """
    s = str(s).strip().lower()
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    s = s.replace(' ', '_')
    return s

def find_best_col(df, candidates):
    """
    Encuentra la mejor coincidencia de nombre de columna en el DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame donde buscar
        candidates (list): Lista de nombres candidatos
        
    Returns:
        str or None: Nombre de columna encontrado o None
    """
    norm_cols = {normalize_col_name(c): c for c in df.columns}
    for cand in candidates:
        n = normalize_col_name(cand)
        if n in norm_cols:
            return norm_cols[n]
    
    # Búsqueda parcial como fallback
    for n, orig in norm_cols.items():
        for cand in candidates:
            if normalize_col_name(cand) in n:
                return orig
    return None

# Ejemplo de uso
print("🔧 Función normalize_col_name:")
print(f"'Precio Unitario' → '{normalize_col_name('Precio Unitario')}'")
print(f"'Categoría' → '{normalize_col_name('Categoría')}'")
print(f"'Total Venta' → '{normalize_col_name('Total Venta')}'")

## 2.2 Funciones de Análisis Estadístico

Funciones especializadas para el cálculo de estadísticas descriptivas y detección de outliers:

In [ ]:
def describe_series(s: pd.Series):
    """
    Calcula estadísticas descriptivas completas para una serie.
    
    Args:
        s (pd.Series): Serie de datos a analizar
        
    Returns:
        dict: Diccionario con estadísticas descriptivas
    """
    return {
        'count': int(s.count()),
        'mean': float(s.mean()),
        'median': float(s.median()),
        'std': float(s.std()),
        'min': float(s.min()),
        'max': float(s.max()),
        'skew': float(s.skew())
    }

def detect_bimodal(s: pd.Series, bins=30):
    """
    Detecta distribuciones bimodales contando picos en el histograma.
    
    Args:
        s (pd.Series): Serie de datos
        bins (int): Número de bins para el histograma
        
    Returns:
        tuple: (es_bimodal, número_de_picos)
    """
    counts, edges = np.histogram(s.dropna(), bins=bins)
    peaks = 0
    for i in range(1, len(counts)-1):
        if counts[i] > counts[i-1] and counts[i] > counts[i+1]:
            peaks += 1
    return peaks >= 2, peaks

def iqr_outliers(s: pd.Series):
    """
    Detecta outliers usando el método del Rango Intercuartílico (IQR).
    
    Args:
        s (pd.Series): Serie de datos
        
    Returns:
        tuple: (máscara_outliers, límite_inferior, límite_superior)
    """
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    mask = (s < low) | (s > high)
    return mask, low, high

def ensure_dirs():
    """Crea los directorios necesarios para guardar resultados."""
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    OUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Funciones de análisis estadístico definidas")

# 3. 📊 Carga y Preparación de Datos

## 3.1 Carga de Archivos Excel

El sistema carga automáticamente todos los archivos necesarios y maneja errores si algún archivo no está disponible:

In [ ]:
# Asegurar que existen los directorios
ensure_dirs()

# Definir rutas de archivos
files = {
    'clientes': DB_DIR / 'clientes.xlsx',
    'detalle_ventas': DB_DIR / 'detalle_ventas.xlsx',
    'productos': DB_DIR / 'productos.xlsx',
    'ventas': DB_DIR / 'ventas.xlsx'
}

# Cargar archivos con manejo de errores
dfs = {}
for key, path in files.items():
    try:
        dfs[key] = pd.read_excel(path, engine='openpyxl')
        print(f"✅ Cargado {path.name}: {dfs[key].shape[0]} registros")
    except FileNotFoundError:
        print(f"⚠️ No se encontró {path.name}")
        dfs[key] = None

# Verificar datos cargados
print(f"\n📊 Resumen de datos cargados:")
for name, df in dfs.items():
    if df is not None:
        print(f"  • {name}: {df.shape[0]} filas × {df.shape[1]} columnas")

## 3.2 Integración de Datos por Categorías

**Característica Clave:** El sistema integra automáticamente la información de productos con las ventas para habilitar análisis por categorías:

In [ ]:
# Seleccionar el DataFrame principal (detalle_ventas preferido)
df_source = dfs.get('detalle_ventas') if dfs.get('detalle_ventas') is not None else dfs.get('ventas')

if df_source is None:
    print("❌ No se encontraron datos de ventas")
    raise SystemExit("No se pueden continuar sin datos de ventas")

# Integración con datos de productos para análisis por categoría
df_productos = dfs.get('productos')
if df_productos is None:
    print("⚠️ Sin datos de productos. Análisis por categoría no estará disponible.")
    df_merged = df_source.copy()
    analyze_categories = False
else:
    print("✅ Integrando datos de productos para análisis por categoría...")
    # Merge con sufijos para evitar conflictos de nombres
    df_merged = pd.merge(df_source, 
                        df_productos[['id_producto', 'categoria', 'nombre_producto']], 
                        on='id_producto', how='left', suffixes=('', '_prod'))
    
    # Usar nombre del producto de la tabla productos (más confiable)
    if 'nombre_producto_prod' in df_merged.columns:
        df_merged['nombre_producto'] = df_merged['nombre_producto_prod']
        df_merged = df_merged.drop('nombre_producto_prod', axis=1)
    
    analyze_categories = True
    print(f"📊 Categorías detectadas: {df_merged['categoria'].unique()}")

# Mostrar información del dataset integrado
print(f"\n📋 Dataset final:")
print(f"  • Registros: {len(df_merged)}")
print(f"  • Columnas: {list(df_merged.columns)}")
if analyze_categories:
    print(f"  • Categorías: {df_merged['categoria'].value_counts().to_dict()}")

# 4. 🔍 Análisis Estadístico Descriptivo

## 4.1 Identificación de Variables Clave

El sistema identifica automáticamente las columnas principales para el análisis:

In [ ]:
# Definir patrones de búsqueda para columnas clave
candidate_price = ['precio_unitario', 'precio', 'price', 'unit_price']
candidate_qty = ['cantidad', 'cantidad_vendida', 'qty', 'quantity']
candidate_total = ['total_venta', 'total', 'importe', 'total_amount']

# Identificar columnas automáticamente
price_col = find_best_col(df_merged, candidate_price)
qty_col = find_best_col(df_merged, candidate_qty)
total_col = find_best_col(df_merged, candidate_total)

# Verificar que se encontraron las columnas necesarias
if not any([price_col, qty_col, total_col]):
    print("❌ No se encontraron columnas esperadas")
    print(f"Columnas disponibles: {list(df_merged.columns)}")
    raise SystemExit("No se pueden continuar sin variables numéricas")

# Crear mapeo de columnas
target_cols = {}
if price_col:
    target_cols['precio_unitario'] = price_col
if qty_col:
    target_cols['cantidad'] = qty_col
if total_col:
    target_cols['total_venta'] = total_col

print("✅ Variables identificadas:")
for label, col in target_cols.items():
    print(f"  • {label} → '{col}'")

## 4.2 Estadísticas Descriptivas Completas

Análisis estadístico completo de las variables principales:

In [ ]:
# Convertir a numérico para asegurar cálculos correctos
for label, col in target_cols.items():
    df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

# Calcular estadísticas descriptivas para cada variable
print("📊 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 50)

for label, col in target_cols.items():
    s = df_merged[col]
    stats = describe_series(s)
    bimodal, peaks = detect_bimodal(s.dropna())
    
    print(f"\n🔹 {label.upper()} (columna: {col})")
    print(f"   Count: {stats['count']}")
    print(f"   Media: {stats['mean']:,.2f}")
    print(f"   Mediana: {stats['median']:,.2f}")
    print(f"   Desv. Est.: {stats['std']:,.2f}")
    print(f"   Min: {stats['min']:,.2f}")
    print(f"   Max: {stats['max']:,.2f}")
    print(f"   Sesgo: {stats['skew']:.3f}")
    print(f"   Picos detectados: {peaks} (¿Bimodal? {bimodal})")

# Crear DataFrame resumen para mejor visualización
summary_data = []
for label, col in target_cols.items():
    s = df_merged[col]
    stats = describe_series(s)
    summary_data.append({
        'Variable': label,
        'Columna': col,
        'Count': stats['count'],
        'Media': round(stats['mean'], 2),
        'Mediana': round(stats['median'], 2),
        'Desv_Est': round(stats['std'], 2),
        'Min': round(stats['min'], 2),
        'Max': round(stats['max'], 2),
        'Sesgo': round(stats['skew'], 3)
    })

df_summary = pd.DataFrame(summary_data)
print(f"\n📋 TABLA RESUMEN:")
print(df_summary.to_string(index=False))

# 5. 📈 Análisis de Correlaciones

## 5.1 Matriz de Correlaciones General

Análisis de correlaciones entre las variables principales del negocio:

In [ ]:
# Crear matriz de correlaciones
num_cols = [c for c in target_cols.values()]
df_corr = df_merged[num_cols].dropna()
corr_matrix = df_corr.corr()

print("🔗 MATRIZ DE CORRELACIONES")
print("=" * 40)
print(corr_matrix)

# Análisis de correlaciones específicas
if 'precio_unitario' in target_cols and 'total_venta' in target_cols:
    r_precio_total = corr_matrix.loc[target_cols['precio_unitario'], target_cols['total_venta']]
    print(f"\n💡 Precio Unitario ↔ Total Venta: r = {r_precio_total:.3f}")
    
    if r_precio_total > 0.7:
        strength = "MUY FUERTE 🔴"
    elif r_precio_total > 0.5:
        strength = "FUERTE 🟡"
    elif r_precio_total > 0.3:
        strength = "MODERADA 🟢"
    else:
        strength = "DÉBIL ⚪"
    print(f"   Interpretación: Correlación {strength}")

if 'cantidad' in target_cols and 'total_venta' in target_cols:
    r_cantidad_total = corr_matrix.loc[target_cols['cantidad'], target_cols['total_venta']]
    print(f"\n💡 Cantidad ↔ Total Venta: r = {r_cantidad_total:.3f}")
    
    if r_cantidad_total > 0.7:
        strength = "MUY FUERTE 🔴"
    elif r_cantidad_total > 0.5:
        strength = "FUERTE 🟡" 
    elif r_cantidad_total > 0.3:
        strength = "MODERADA 🟢"
    else:
        strength = "DÉBIL ⚪"
    print(f"   Interpretación: Correlación {strength}")

# Visualizar la matriz de correlaciones
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=2, cbar_kws={"shrink": .8})
plt.title('🔗 Matriz de Correlaciones - Variables Principales', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# 6. 🏷️ Análisis por Categorías de Producto

Esta es una de las características más avanzadas del sistema, que permite segmentar todos los análisis por categoría de producto.

In [ ]:
def analyze_by_category(df_merged, target_cols):
    """
    Función principal para análisis por categorías.
    
    Returns:
        dict: Estadísticas por categoría
    """
    if not analyze_categories:
        print("⚠️ Análisis por categorías no disponible")
        return None
    
    # Estadísticas por categoría
    category_stats = {}
    categories = df_merged['categoria'].unique()
    
    print("🏷️ ANÁLISIS POR CATEGORÍA DE PRODUCTO")
    print("=" * 60)
    
    for categoria in categories:
        df_cat = df_merged[df_merged['categoria'] == categoria]
        
        stats = {
            'transacciones': len(df_cat),
            'productos_unicos': df_cat['id_producto'].nunique(),
            'ingresos_totales': df_cat[target_cols['total_venta']].sum(),
            'precio_promedio': df_cat[target_cols['precio_unitario']].mean(),
            'cantidad_promedio': df_cat[target_cols['cantidad']].mean(),
            'ticket_promedio': df_cat[target_cols['total_venta']].mean()
        }
        category_stats[categoria] = stats
        
        print(f'\n📊 CATEGORÍA: {categoria.upper()}')
        print(f"   • Transacciones: {stats['transacciones']:,}")
        print(f"   • Productos únicos: {stats['productos_unicos']}")
        print(f"   • Ingresos totales: ${stats['ingresos_totales']:,.2f}")
        print(f"   • Precio unitario promedio: ${stats['precio_promedio']:.2f}")
        print(f"   • Cantidad promedio: {stats['cantidad_promedio']:.1f}")
        print(f"   • Ticket promedio: ${stats['ticket_promedio']:.2f}")
    
    return category_stats

# Ejecutar análisis por categorías
category_stats = analyze_by_category(df_merged, target_cols)

if category_stats:
    # Ranking por ingresos
    ranking = sorted(category_stats.items(), key=lambda x: x[1]['ingresos_totales'], reverse=True)
    print(f'\n🏆 RANKING POR INGRESOS TOTALES:')
    for i, (cat, stats) in enumerate(ranking, 1):
        total_ingresos = sum(s['ingresos_totales'] for _, s in category_stats.items())
        percentage = (stats['ingresos_totales'] / total_ingresos) * 100
        print(f"   {i}. {cat}: ${stats['ingresos_totales']:,.2f} ({percentage:.1f}%)")

# 7. 🎨 Visualizaciones Avanzadas

## 7.1 Top 10 Productos por Venta Total

Gráfico que muestra los productos más exitosos con ID + nombre:

In [ ]:
# Configuración de estilo
sns.set_style("whitegrid")
plt.rcParams['figure.facecolor'] = 'white'

# Top 10 productos por venta total
if 'total_venta' in target_cols:
    # Calcular top 10 con nombres
    if 'nombre_producto' in df_merged.columns:
        top_10_products = df_merged.groupby('id_producto').agg({
            target_cols['total_venta']: 'sum',
            'nombre_producto': 'first'
        }).nlargest(10, target_cols['total_venta'])
        
        # Crear etiquetas mejoradas
        labels = []
        for idx, name in zip(top_10_products.index, top_10_products['nombre_producto']):
            short_name = name[:25] + "..." if len(name) > 25 else name
            labels.append(f'{idx}\n{short_name}')
    else:
        top_10_products = df_merged.groupby('id_producto').agg({
            target_cols['total_venta']: 'sum'
        }).nlargest(10, target_cols['total_venta'])
        labels = [f'ID: {idx}' for idx in top_10_products.index]
    
    # Crear gráfico
    plt.figure(figsize=(16, 8))
    colors = plt.cm.Set3(range(len(top_10_products)))
    bars = plt.bar(range(len(top_10_products)), 
                   top_10_products[target_cols['total_venta']], 
                   color=colors, alpha=0.8, edgecolor='black', linewidth=1)
    
    # Etiquetas y valores
    plt.xticks(range(len(top_10_products)), labels, rotation=45, ha='right', fontsize=10)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'${height:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    plt.title('🏆 Top 10 Productos por Venta Total\n(ID + Nombre del Producto)', 
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('Producto (ID + Nombre)', fontsize=12, fontweight='600')
    plt.ylabel('Total de Ventas ($)', fontsize=12, fontweight='600')
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    # Mostrar datos tabulares
    print("\n🏆 TOP 10 PRODUCTOS POR VENTA TOTAL:")
    print("-" * 60)
    for i, (idx, row) in enumerate(top_10_products.iterrows(), 1):
        name = row.get('nombre_producto', f'Producto {idx}')
        venta = row[target_cols['total_venta']]
        print(f"{i:2d}. ID {idx}: {name}")
        print(f"    💰 ${venta:,.2f}")
        print()

# 8. 📊 Exportación de Resultados

## 8.1 Generación de Reporte Final

El sistema genera automáticamente un reporte completo con todos los análisis:

In [ ]:
# Generar reporte final
report_lines = []
report_lines.append('ANÁLISIS DE VENTAS CON CATEGORÍAS DE PRODUCTO')
report_lines.append('=' * 50)
report_lines.append(f'Total de registros analizados: {len(df_merged)}')
if analyze_categories:
    report_lines.append(f'Categorías de producto: {", ".join(df_merged["categoria"].unique())}')

# Agregar estadísticas descriptivas al reporte
report_lines.append('\nESTADÍSTICAS DESCRIPTIVAS:')
for label, col in target_cols.items():
    s = df_merged[col]
    stats = describe_series(s)
    report_lines.append(f"\n{label.upper()} (columna: {col})")
    for k, v in stats.items():
        report_lines.append(f"  {k}: {v}")

# Agregar correlaciones al reporte
report_lines.append('\nCORRELACIONES PRINCIPALES:')
if len(target_cols) >= 2:
    for i, (label1, col1) in enumerate(target_cols.items()):
        for label2, col2 in list(target_cols.items())[i+1:]:
            corr_val = df_merged[[col1, col2]].corr().iloc[0, 1]
            report_lines.append(f"  {label1} ↔ {label2}: r = {corr_val:.3f}")

# Detección de outliers
report_lines.append('\nOUTLIERS DETECTADOS (método IQR):')
for label, col in target_cols.items():
    s = df_merged[col]
    mask, low, high = iqr_outliers(s.dropna())
    outliers = s.dropna()[mask]
    report_lines.append(f"  {label}: {len(outliers)} outliers detectados")
    if len(outliers) > 0:
        report_lines.append(f"    Rango normal: {low:.2f} - {high:.2f}")

# Guardar reporte
report_path = OUT_DIR / 'reporte_analisis_notebook.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(report_lines))

print(f"✅ Reporte guardado en: {report_path}")
print(f"📁 Líneas del reporte: {len(report_lines)}")

# Mostrar resumen final
print("\n" + "="*60)
print("🎯 RESUMEN EJECUTIVO")
print("="*60)
print(f"📊 Registros analizados: {len(df_merged):,}")
if analyze_categories:
    print(f"🏷️  Categorías: {len(df_merged['categoria'].unique())}")
print(f"📈 Variables analizadas: {len(target_cols)}")
print(f"📁 Archivos generados: report, múltiples gráficos")
print(f"✅ Análisis completado exitosamente")

# 📋 Resumen de Funcionalidades Documentadas

## ✅ **Funcionalidades Principales Incluidas:**

### 🔧 **Funciones Utilitarias**
- `normalize_col_name()` → Normalización de nombres de columnas
- `find_best_col()` → Búsqueda inteligente de columnas
- `describe_series()` → Estadísticas descriptivas completas
- `detect_bimodal()` → Detección de distribuciones bimodales
- `iqr_outliers()` → Detección de outliers por IQR

### 📊 **Análisis Avanzados**
- **Estadísticas Descriptivas** → Count, media, mediana, desv. std., min, max, sesgo
- **Matriz de Correlaciones** → Análisis de relaciones entre variables
- **Análisis por Categorías** → Segmentación por Alimentos, Bebidas, Limpieza
- **Detección de Outliers** → Método IQR con límites automáticos

### 🎨 **Visualizaciones**
- **Top 10 Productos** → Con ID + nombres de productos
- **Heatmap de Correlaciones** → Matriz visual de correlaciones
- **Análisis por Categorías** → Múltiples gráficos segmentados
- **Boxplots Comparativos** → Distribuciones por categoría

### 🔄 **Automatización**
- **Carga Automática** → De todos los archivos Excel
- **Integración de Datos** → Merge automático con manejo de conflictos
- **Generación de Reportes** → Archivo de texto con resultados
- **Manejo de Errores** → Validaciones y mensajes informativos

## 🚀 **Ventajas del Sistema Documentado:**

1. **Modular** → Funciones independientes y reutilizables
2. **Robusto** → Manejo de errores y validaciones
3. **Escalable** → Fácil agregar nuevas categorías/análisis
4. **Automatizado** → Proceso completo sin intervención manual
5. **Interpretable** → Resultados claros y accionables

---

**📝 Nota:** Este notebook sirve como documentación completa del sistema de análisis. Cada celda puede ejecutarse independientemente para explorar funcionalidades específicas.